In [2]:
!which python

/Users/simon.hughes/anaconda/envs/phd_py36/bin/python


In [1]:
from Settings import Settings
import dill

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
# NOTE: These predictions are generated from the "./notebooks/SEARN/Keras - Train Tagger and Save CV Predictions For Word Tags.ipynb" notebook
# used as inputs to parsing model
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

train_fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [1]:
from BrattEssay import load_bratt_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

settings = Settings()

#essays = load_bratt_essays(settings.data_directory + "SkinCancer/EBA1415_Merged/")
folder = settings.data_directory + "CoralBleaching/BrattData/EBA1415_Merged/"
essays = load_bratt_essays(folder)

wd_sent_freq = defaultdict(int)
all_codes = set()
#Stores all words for the spelling corrector
words = []
all_sentences = []
sentencesForCode = defaultdict(list)
for essay in essays:
    for sentence in essay.tagged_sentences:
        wdsInSent = set()
        codes4sentence = set()
        sent = []
        for w, tags in sentence:
            words.append(w)
            all_codes.update(tags)
            codes4sentence.update(tags)
            if w not in wdsInSent:
                wdsInSent.add(w)
                wd_sent_freq[w] += 1
            sent.append(w)
        all_sentences.append(sent)
        for code in codes4sentence:
            sentencesForCode[code].append(sentence)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
(1154, 'files found')
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_AEKD_4_CB_ES-05571.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_AEKD_4_CB_ES-05904.ann file as .txt file is no essay'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_BGJD_1_CB_ES-05733.ann file as .txt file is no essay //'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_ERSK_7_CB_ES-05798.ann file as .txt file is no essay //'
Skipping /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/BrattData/EBA1415_Merged/EBA1415_KYLS_5_CB_ES-05671.a

### Compute Stats over the Essays

In [46]:
from collections import defaultdict

def get_tag_freq(train_tagged_essays):
    stag_freq = defaultdict(int)
    unique_words = set()
    for essay in train_tagged_essays:
        for sentence in essay.sentences:
            for word, tags in sentence:
                for tag in tags:
                    stag_freq[tag] += 1
    return stag_freq


def get_tag_2_sent(train_tagged_essays):
    tag2sent = defaultdict(list)
    sent2tags = defaultdict(set)
    sent_ix = 0
    for essay in train_tagged_essays:
        for sentence in essay.sentences:
            sent_ix += 1
            unique_sent_tags = set()
            sent_words = []
            for word, tags in sentence:
                sent_words.append(word)
                for tag in tags:
                    unique_sent_tags.add(tag)
        
            if unique_sent_tags and sent_words:
                str_sent = str(sent_ix) + ": " + " ".join(sent_words)
                sent2tags[str_sent] = unique_sent_tags
                for tag in unique_sent_tags:
                    tag2sent[tag].append(str_sent)                    
    return tag2sent, sent2tags

def get_cr_tags(train_tagged_essays, tag_essays_test):

    tag_freq = get_tag_freq(train_tagged_essays, tag_essays_test)
    crel_tags = list((t for t in tag_freq.keys() if ("->" in t) and
                    not "Anaphor" in t and
                    not "other" in t and
                    not "rhetorical" in t and
                    not "factor" in t and
                    1 == 1
                    ))
    return crel_tags

def is_cr_tag(t):    
    return ("->" in t) and \
        not "Anaphor" in t and \
        not "other" in t and \
        not "rhetorical" in t and \
        not "factor" in t

In [47]:
tag2sent, sent2tags = get_tag_2_sent(pred_tagged_essays_train)

In [48]:
tag_freq = get_tag_freq(pred_tagged_essays_train)
cr_tags = [t for t in tag_freq.keys() if is_cr_tag(t)]
#cr_tags

In [49]:
def get_code_from_cr_part(cr_part):
    return cr_part.lower().strip().replace("causer:","").replace("result:","")

In [50]:
same_code_crs = set()
for cr_tag in cr_tags:
    left,right = cr_tag.split("->")
    left = get_code_from_cr_part(left)
    right = get_code_from_cr_part(right)
    if left == right:
        same_code_crs.add(cr_tag)
same_code_crs

{'Causer:11->Result:11', 'Causer:50->Result:50'}

In [62]:
for cr_tag in same_code_crs:
    print("*" * 80)
    print(cr_tag)
    for i, sent in enumerate(tag2sent[cr_tag]):        
        tags = sent2tags[sent]
        cr_tags = [t for t in tags if is_cr_tag(t)]
        print(cr_tags)
        print(sent)
        print()

********************************************************************************
Causer:11->Result:11
['Causer:11->Result:11', 'Causer:3->Result:4']
2738: balance between co2 and water temperature is also threaten by extreme storms .

********************************************************************************
Causer:50->Result:50
['Causer:1->Result:50', 'Causer:50->Result:50']
190: if the shifting trade winds make colors the corals will die and cause coral bleaching .

['Causer:13->Result:50', 'Causer:50->Result:50']
1859: if there isnt enough salinity need for coral health , they will die and change into white color .

['Causer:5b->Result:50', 'Causer:50->Result:50']
1942: it is 00 - 00 percent of their energy intake so the less energy easier for it to die and turn white .



In [55]:
list(sent2tags.items())[0:10]

[('1: what leads to differences in the rates of coral bleaching .', {'50'}),
 ('3: coral bleaching shows bleaching and healthy bleaching event that makes coral vulnerable to disease and starvation .',
  {'50'}),
 ('4: coral bleaching is almost noticeable in the pacific ocean .',
  {'50', 'COMPILED'}),
 ('5: the part of coral called " zooanthellae " are not getting sunlight .',
  {'5'}),
 ('6: and if they dont get or much sunlight they start to loses their color .',
  {'5',
   '50',
   'Anaphor',
   'Causer',
   'Causer:5',
   'Causer:5->Result:50',
   'Result',
   'Result:50',
   'explicit'}),
 ('7: the reason why is because the " zooanthellae . " if not getting mixed up with carbon dioxide .',
  {'4'}),
 ('9: also its a threats for us because means that more extreme storms are most likely to occurs .',
  {'11'}),
 ('10: also the water us getting to salty .', {'13', 'COMPILED'}),
 ('11: what leads to different in the rates of coral bleaching is that INFREQUENT different types of INFREQ